Learning Rate Scheduling

In [1]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
[name for name in dir(keras.optimizers.schedules)if not name.startswith('_')]

['CosineDecay',
 'CosineDecayRestarts',
 'ExponentialDecay',
 'InverseTimeDecay',
 'LearningRateSchedule',
 'PiecewiseConstantDecay',
 'PolynomialDecay',
 'deserialize',
 'serialize']

1. Polynomial Decay

In [3]:
lrPoly = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=(.1),
    decay_steps=1000,
    end_learning_rate=0.0001,
    power=0.5,
    cycle=False
)

2. Exponential Decay

In [4]:
lrExp = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.1, decay_steps=100000, decay_rate=.96)

3. Piecewise Constant Decay

In [5]:
lrPcd = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=[100000,110000], values=[1.0, 0.5, 0.1])

Ex

In [6]:
#Load data
from tensorflow.keras.datasets import fashion_mnist
(xTrainSet, yTrainSet), (xTest, yTest) = fashion_mnist.load_data()

#Split data
from sklearn.model_selection import train_test_split
xTrain, xValid ,yTrain, yValid = train_test_split(xTrainSet, yTrainSet, random_state=1)

#Preprocessing
pixelMeans = xTrain.mean(axis=0, keepdims=True)
pixelStds = xTrain.std(axis=0, keepdims=True)
xTrainScaled = (xTrain - pixelMeans) / pixelStds
xValidScaled = (xValid - pixelMeans) / pixelStds
xTestScaled = (xTest - pixelMeans) / pixelStds

In [7]:
tf.keras.backend.clear_session()
np.random.seed(1)
tf.random.set_seed(1)

In [8]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(300, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.Dense(100, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.Dense(10, activation='softmax')
])

In [9]:
lrExp = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=.1, decay_steps=100000, decay_rate=.96)

model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=lrExp), metrics=['accuracy'])

In [10]:
train = model.fit(xTrainScaled, yTrain, epochs=2, validation_data=(xValidScaled, yValid))

Epoch 1/2


2024-02-04 13:13:00.486406: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1407/1407 [==============================] - 2s 1ms/step - loss: 0.5141 - accuracy: 0.8204 - val_loss: 0.6907 - val_accuracy: 0.7621
Epoch 2/2
1407/1407 [==============================] - 1s 931us/step - loss: 0.3797 - accuracy: 0.8611 - val_loss: 0.4455 - val_accuracy: 0.8477
